In [1]:
# After you have built and executed a compute graph
# on https://demo.kurve.ai load the resultant
# dataset here.

In [2]:
# we'll be focused on the user badge problem first
# https://relbench.stanford.edu/datasets/rel-stack/#user-badge

In [59]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import catboost
from torch_frame.utils import infer_df_stype

In [60]:
pd.set_option('display.max_columns', 1000)

In [61]:
# overwrite with your dataset
df = pd.read_parquet("s3://kurve-customers/1d0ab129-e9c9-4ea8-8282-040501ce0e6e/31/output/user_badge_train")

In [62]:
df.shape

(333784, 115)

In [63]:
df.head()

,User_Id,User_AccountId,User_DisplayName,User_Location,User_ProfileImageUrl,User_WebsiteUrl,User_AboutMe,User_CreationDate,User_LastAccessDate,Posts_Id_count,Posts_OwnerUserId_count,Posts_LastEditorUserId_count,Posts_PostTypeId_count,Posts_AcceptedAnswerId_count,Posts_ParentId_count,Posts_ContentLicense_count,Posts_FavoriteCount_avg,Posts_FavoriteCount_sum,Posts_FavoriteCount_min,Posts_FavoriteCount_max,Posts_CreationDate_min,Posts_CreationDate_max,Posts_CommunityOwnedDate_min,Posts_CommunityOwnedDate_max,Posts_ClosedDate_min,Posts_ClosedDate_max,Posts_LastEditDate_min,Posts_LastEditDate_max,Posts_LastActivityDate_min,Posts_LastActivityDate_max,Vote_Id_count_avg,Vote_Id_count_sum,Vote_Id_count_min,Vote_Id_count_max,Vote_PostId_count_avg,Vote_PostId_count_sum,Vote_PostId_count_min,Vote_PostId_count_max,Vote_VoteTypeId_count_avg,Vote_VoteTypeId_count_sum,Vote_VoteTypeId_count_min,Vote_VoteTypeId_count_max,Vote_CreationDate_min_min,Vote_CreationDate_min_max,Vote_CreationDate_max_min,Vote_CreationDate_max_max,Vote_num_events_30d_avg,Vote_num_events_30d_sum,Vote_num_events_30d_min,Vote_num_events_30d_max,Vote_num_events_60d_avg,Vote_num_events_60d_sum,Vote_num_events_60d_min,Vote_num_events_60d_max,Vote_num_events_90d_avg,Vote_num_events_90d_sum,Vote_num_events_90d_min,Vote_num_events_90d_max,Vote_num_events_180d_avg,Vote_num_events_180d_sum,Vote_num_events_180d_min,Vote_num_events_180d_max,Vote_num_events_365d_avg,Vote_num_events_365d_sum,Vote_num_events_365d_min,Vote_num_events_365d_max,Vote_num_events_730d_avg,Vote_num_events_730d_sum,Vote_num_events_730d_min,Vote_num_events_730d_max,Posts_num_events_30d,Posts_num_events_60d,Posts_num_events_90d,Posts_num_events_180d,Posts_num_events_365d,Posts_num_events_730d,Comm_Id_count,Comm_PostId_count,Comm_UserId_count,Comm_ContentLicense_count,Comm_CreationDate_min,Comm_CreationDate_max,Comm_num_events_30d,Comm_num_events_60d,Comm_num_events_90d,Comm_num_events_180d,Comm_num_events_365d,Comm_num_events_730d,Badg_Id_count,Badg_UserId_count,Badg_Class_count,Badg_TagBased_sum,Badg_Date_min,Badg_Date_max,Badg_num_events_30d,Badg_num_events_60d,Badg_num_events_90d,Badg_num_events_180d,Badg_num_events_365d,Badg_num_events_730d,Badg_Id_label,Post_Id_count,Post_PostId_count,Post_UserId_count,Post_PostHistoryTypeId_count,Post_ContentLicense_count,Post_RevisionGUID_count,Post_CreationDate_min,Post_CreationDate_max,Post_num_events_30d,Post_num_events_60d,Post_num_events_90d,Post_num_events_180d,Post_num_events_365d,Post_num_events_730d
0,89987,131921,stefan.s,"Solingen, Germany",None,https://stopnowapp.wordpress.com/,<p>I am responsible for the User Interface Dev...,2015-09-21 07:27:39.680,2016-07-26 11:11:55.303,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,3,3,3,0.0,2015-09-21 07:30:49.073,2015-09-21 08:39:09.953,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,90087,1369195,Bardo91,"Seville, Spain",None,http://elescritoriodetesla.blogspot.com.es/,<p>A motivated student of Engineering of the U...,2015-09-22 07:52:34.613,2022-07-10 09:34:07.930,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,4,4,0.0,2015-09-22 07:57:01.553,2015-11-14 15:37:07.037,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,90474,3886820,Rushik Thumar,"Ahmedabad, India",None,None,None,2015-09-26 10:01:19.210,2018-08-24 20:38:33.327,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN

In [64]:
# get the label
[c for c in df.columns if 'label' in c]

['Badg_Id_label']

In [65]:
label = [c for c in df.columns if 'label' in c][0]
df[label]

0         <NA>
1         <NA>
2         <NA>
3         <NA>
4         <NA>
          ... 
333779       1
333780       1
333781       2
333782       1
333783       1
Name: Badg_Id_label, Length: 333784, dtype: Int64

In [66]:
# transform the label to a binary 
df['badge_label'] = df[label].apply(lambda x: 1 if not pd.isnull(x) else 0)


In [67]:
df.badge_label.sum()/len(df)

0.027877309877046234

In [68]:
class_weights = [1, round((1.0/(df.badge_label.sum()/len(df))))]
print(class_weights)

[1, 36]


In [69]:
train, test = train_test_split(df)

In [70]:
train.shape

(250338, 116)

In [71]:
test.shape

(83446, 116)

In [72]:
# get the features
stypes = infer_df_stype(df)

In [73]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and not k.startswith('User_')
    and 'label' not in k]

In [74]:
print(len(features))
print(features)


85
['Posts_Id_count', 'Posts_OwnerUserId_count', 'Posts_LastEditorUserId_count', 'Posts_PostTypeId_count', 'Posts_AcceptedAnswerId_count', 'Posts_ParentId_count', 'Posts_ContentLicense_count', 'Posts_FavoriteCount_avg', 'Posts_FavoriteCount_sum', 'Posts_FavoriteCount_min', 'Posts_FavoriteCount_max', 'Vote_Id_count_avg', 'Vote_Id_count_sum', 'Vote_Id_count_min', 'Vote_Id_count_max', 'Vote_PostId_count_avg', 'Vote_PostId_count_sum', 'Vote_PostId_count_min', 'Vote_PostId_count_max', 'Vote_VoteTypeId_count_avg', 'Vote_VoteTypeId_count_sum', 'Vote_VoteTypeId_count_min', 'Vote_VoteTypeId_count_max', 'Vote_num_events_30d_avg', 'Vote_num_events_30d_sum', 'Vote_num_events_30d_min', 'Vote_num_events_30d_max', 'Vote_num_events_60d_avg', 'Vote_num_events_60d_sum', 'Vote_num_events_60d_min', 'Vote_num_events_60d_max', 'Vote_num_events_90d_avg', 'Vote_num_events_90d_sum', 'Vote_num_events_90d_min', 'Vote_num_events_90d_max', 'Vote_num_events_180d_avg', 'Vote_num_events_180d_sum', 'Vote_num_events_18

In [75]:
mdl = catboost.CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    depth=7,
    loss_function='Logloss',
    eval_metric='AUC',
    class_weights=class_weights,
    boosting_type='Ordered',
    subsample=0.9,
    l2_leaf_reg=5,
    bagging_temperature=5,
    border_count=10,
    feature_border_type='GreedyLogSum',
    verbose=100
)

In [76]:
mdl.fit(train[features], train['badge_label'])

0:	total: 94.7ms	remaining: 9.37s
99:	total: 9.24s	remaining: 0us


In [77]:
test['pred'] = mdl.predict(test[features])

In [78]:
print(metrics.roc_auc_score(test['badge_label'], test['pred']))
print(metrics.accuracy_score(test['badge_label'], test['pred']))


0.6935054015029571
0.8506339429091868


In [79]:
list(reversed(sorted(zip(mdl.feature_names_, mdl.feature_importances_), key=lambda x: x[1])))[0:10]

[('Badg_num_events_365d', 5.243828421500643),
 ('Posts_PostTypeId_count', 5.058428047047275),
 ('Badg_num_events_90d', 5.021733823675838),
 ('Badg_num_events_180d', 4.607058280547995),
 ('Posts_Id_count', 3.7870813086945674),
 ('Badg_num_events_730d', 3.7002736120287407),
 ('Badg_num_events_60d', 3.6630208785600353),
 ('Vote_num_events_730d_max', 3.1481667932868476),
 ('Badg_TagBased_sum', 2.8360537933072796),
 ('Badg_Id_count', 2.7717972812360374)]

In [80]:
print(metrics.classification_report(test['badge_label'],test['pred']))


              precision    recall  f1-score   support

           0       0.98      0.86      0.92     81122
           1       0.10      0.53      0.16      2324

    accuracy                           0.85     83446
   macro avg       0.54      0.69      0.54     83446
weighted avg       0.96      0.85      0.90     83446

